In [1]:
import os

In [2]:
%pwd

'c:\\Users\\bayir\\Documents\\Kidney_Disease_Detection - Copie\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\bayir\\Documents\\Kidney_Disease_Detection - Copie'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from CnnClassifier.constants import *
from CnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [18]:
import gdown
import os
import zipfile
from CnnClassifier import logger
from CnnClassifier.utils.common import get_size

In [21]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'nazmul0087/ct-kidney-dataset-normal-cyst-tumor-and-stone'
            gdown.download(prefix+file_id,zip_download_dir, unzip=True)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception("Error occurred during data ingestion process.")
    raise e


[2025-04-27 12:30:35,431: INFO: common: created directory at: artifacts]
[2025-04-27 12:30:35,433: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-27 12:30:35,434: INFO: 3882145863: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2025-04-27 12:30:35,435: ERROR: 2628198839: Error occurred during data ingestion process.]
Traceback (most recent call last):
  File "C:\Users\bayir\AppData\Local\Temp\ipykernel_34728\2628198839.py", line 5, in <module>
    data_ingestion.download_file()
  File "C:\Users\bayir\AppData\Local\Temp\ipykernel_34728\3882145863.py", line 24, in download_file
    raise e
  File "C:\Users\bayir\AppData\Local\Temp\ipykernel_34728\3882145863.py", line 19, in download_file
    gdown.download(prefix+file_id,zip_download_dir, unzip=True)
TypeError: download() got an unexpected keyword argument 'unzip'


TypeError: download() got an unexpected keyword argument 'unzip'

In [34]:
import os
import zipfile
import kaggle

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from Kaggle
        '''
        try: 
            dataset_name = self.config.source_URL  # should be in the format "owner/dataset-name"
            zip_download_dir = self.config.local_data_file
            os.makedirs(os.path.dirname(zip_download_dir), exist_ok=True)
            logger.info(f"Downloading dataset {dataset_name} into file {zip_download_dir}")

            kaggle.api.dataset_download_files(
            'nazmul0087/ct-kidney-dataset-normal-cyst-tumor-and-stone',
            path='artifacts/data_ingestion/',
            unzip=True)

            logger.info(f"Downloaded dataset {dataset_name} into {zip_download_dir}")

        except Exception as e:
            raise e

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()

except Exception as e:
    logger.exception("Error occurred during data ingestion process.")
    raise e



[2025-04-27 12:49:05,433: INFO: common: created directory at: artifacts]
[2025-04-27 12:49:05,435: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-27 12:49:05,438: INFO: 3765667004: Downloading dataset https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
Dataset URL: https://www.kaggle.com/datasets/nazmul0087/ct-kidney-dataset-normal-cyst-tumor-and-stone
[2025-04-27 12:57:18,726: INFO: 3765667004: Downloaded dataset https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into artifacts/data_ingestion/data.zip]
